In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
%run "../Includes/sas alternative"

In [0]:
df = spark.read.parquet(f"{Presentation_path}/Race_Results_R")
df.display()

In [0]:
# identify total points based on race year, team
# whomever having position value = 1 then, True take count of the positions having true, rename it as wins

from pyspark.sql.functions import sum,count,when,col
df_Total_Points = df.groupBy('race_year','team','points','position').agg(sum('points').alias("total_points"),count(when(col('position')==1, True)).alias("Wins"))

In [0]:
df_Total_Points.display()

In [0]:
# identify the rank based up on partitionBY(race_year) order by wins and total points descending order (window functions)
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

window_spec = Window.partitionBy('race_year').orderBy(desc('Wins'), desc('total_points'))

df_rank = df_Total_Points.withColumn("Rank", rank().over(window_spec))

df_rank.display()

In [0]:
# print(race_year==2020) write constructer folder in presentation
df_rank.filter(col('race_year')==2020).display()

In [0]:
df_rank.write.mode('overwrite').parquet(f"{Presentation_path}/Constructer")

In [0]:
dbutils.notebook.exit("Sucessed")